In [ ]:
IAM_URL = "https://iam.cloud.ibm.com"
DATAPLATFORM_URL = "https://api.dataplatform.cloud.ibm.com"
#DATAPLATFORM_URL = "https://api.eu-de.dataplatform.cloud.ibm.com"
#DATAPLATFORM_URL = "https://api.au-syd.dataplatform.cloud.ibm.com"
SERVICE_URL = "https://aiopenscale.cloud.ibm.com"
#SERVICE_URL = "https://au-syd.aiopenscale.cloud.ibm.com"
CLOUD_API_KEY = "<apikey>" # YOUR_CLOUD_API_KEY
WML_CREDENTIALS = {
                "url": "https://us-south.ml.cloud.ibm.com",
                "apikey": CLOUD_API_KEY,
                "auth_url": IAM_URL,
                "wml_location" : "cloud"
}


In [ ]:
import json
from ibm_watsonx_ai import APIClient

wml_client = APIClient(WML_CREDENTIALS)
wml_client.version

## Function to create the access token

This function generates an IAM access token using the provided credentials. The API calls for creating and scoring prompt template assets utilize the token generated by this function.

In [ ]:
import requests, json
def generate_access_token():
    headers={}
    headers["Content-Type"] = "application/x-www-form-urlencoded"
    headers["Accept"] = "application/json"
    data = {
        "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
        "apikey": CLOUD_API_KEY,
        "response_type": "cloud_iam"
    }
    response = requests.post(IAM_URL + "/identity/token", data=data, headers=headers)
    json_data = response.json()
    iam_access_token = json_data["access_token"]
        
    return iam_access_token

iam_access_token = generate_access_token()
print(iam_access_token)

In [ ]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator, CloudPakForDataAuthenticator

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *

service_instance_id = None # Update this to refer to a particular service instance
authenticator = IAMAuthenticator(
    apikey=CLOUD_API_KEY,
    url=IAM_URL
)
wos_client = APIClient(
    authenticator=authenticator,
    service_url=SERVICE_URL,
    service_instance_id=service_instance_id
)
data_mart_id = wos_client.service_instance_id
print(wos_client.version)

# Create Custom Monitor for User Feedback <a name="ptaspace"></a>



In [ ]:
monitor_name = 'user_feedback_metrics'


wos_client.monitor_definitions.delete(monitor_definition_id = monitor_name, background_mode = True, force = True)

In [ ]:
wos_client.monitor_definitions.show()

In [ ]:
def get_definition(monitor_name):
    monitor_definitions = wos_client.monitor_definitions.list().result.monitor_definitions
   
    for definition in monitor_definitions:
        if monitor_name == definition.entity.name:
            return definition
   
    return None
 
 


monitor_name="user_feedback_metrics"
metrics = [MonitorMetricRequest(name='positive feedback',
                                thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=0.8),]), 
          MonitorMetricRequest(name='negative feedback',
                                thresholds=[MetricThreshold(type=MetricThresholdTypes.UPPER_LIMIT, default=0.5)])]
tags = [MonitorTagRequest(name='comment', description='user comment')]
 
existing_definition = get_definition(monitor_name)
 
if existing_definition is None:
    custom_monitor_details = wos_client.monitor_definitions.add(name=monitor_name, metrics=metrics, tags=tags, background_mode=False).result
else:
    custom_monitor_details = existing_definition


In [ ]:
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import IntegratedSystems, ApplicabilitySelection
CUSTOM_METRICS_PROVIDER_CREDENTIALS = {
    "auth_type":"bearer",
    "token_info": {
       "url":  IAM_URL,
       "headers": { "Content-type": "application/x-www-form-urlencoded" }, # update the headers here 
       "payload": "grant_type=urn:ibm:params:oauth:grant-type:apikey&response_type=cloud_iam&apikey="+CLOUD_API_KEY, # update the payload here 
       "method": "POST" # # update the http method here 
    }        
}
custom_metrics_integrated_system = IntegratedSystems(wos_client).add(
    name=monitor_name,
    description=monitor_name,
    type="custom_metrics_provider",
    credentials= CUSTOM_METRICS_PROVIDER_CREDENTIALS,
    connection={
        "display_name": monitor_name,
        #"endpoint": "http://test"
    }
).result

integrated_system_id = custom_metrics_integrated_system.metadata.id
print(integrated_system_id)

In [ ]:
custom_monitor_id = custom_monitor_details.metadata.id
 
print(custom_monitor_id)

In [ ]:
custom_monitor_details = wos_client.monitor_definitions.get(monitor_definition_id=custom_monitor_id).result
print('Monitor definition details:', custom_monitor_details)


In [ ]:
print("integrated_system_id" ,integrated_system_id)
print("custom_monitor_id", custom_monitor_id)